In [4]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np
 
true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])
 
print(round(ndcg_score(
    true, relevance), 2))

0.97
